In [1]:
import os
import random
import re
import requests
from dotenv import load_dotenv

class NewsBot:
    exit_commands = ("quit", "pause", "exit", "goodbye", "bye", "later")
    yes_commands = ("sure", "yes", "absolutely", "yep", "ok", "okay", "ya", "yup", "yea", "yeah", "yah", "ye", "y")
    no_commands = ("no", "nope", "nah", "nay", "n")

    def __init__(self):
        load_dotenv()
        self.api_key = os.getenv("OPENAI_API_KEY")
        if self.api_key is None:
            raise ValueError("OpenAI API key not found. Make sure it's set in the .env file.")
        self.news_channels = {
            'bbc': f'https://newsapi.org/v2/top-headlines?sources=bbc-news&apiKey={self.api_key}',
            'cnn': f'https://newsapi.org/v2/top-headlines?sources=cnn&apiKey={self.api_key}',
            'reuters': f'https://newsapi.org/v2/top-headlines?sources=reuters&apiKey={self.api_key}',
            'aljazeera': f'https://newsapi.org/v2/top-headlines?sources=al-jazeera-english&apiKey={self.api_key}'
        }
        self.news_channel = None

    def add_news_channel(self, channel_name, channel_api_url):
        self.news_channels[channel_name.lower()] = channel_api_url

    def greet(self):
        print("🗞️  Welcome to NewsBot! I can fetch top headlines from different news channels.")
        self.name = input("What is your name?\n")
        reply = input(f"Hello, {self.name}😀! Would you like to get news headlines for today?\n").lower()
        if reply in self.yes_commands:
            self.choose_news_channel()
        else:
            print("Goodbye! Have a nice day.")

    def choose_news_channel(self):
        reply = input("Would you like to get headlines from BBC, CNN or Reuters? If yes, enter the news channel you prefer👇🏽\n").lower()
        if reply in self.news_channels:
            self.news_channel = reply
            self.fetch_news()
        elif reply in self.no_commands:
            reply = input("Would you like to get news headlines from your preferred news channel?😀(yes/no)\n").lower()
            if reply in self.yes_commands:
                self.news_channel = input("Please enter the name of your preferred news channel: \n").lower()
                if self.news_channel not in self.news_channels:
                    self.add_news_channel(self.news_channel, f"https://newsapi.org/v2/top-headlines?sources={self.news_channel}&apiKey={self.api_key}")
                self.fetch_news()
            else:
                print("Goodbye! Have a nice day.")
        else:
            print("🔴 Please choose a valid news channel")
            self.choose_news_channel()

    def make_exit(self, reply):
        return reply.lower() in self.exit_commands

    def fetch_news(self):
        if self.news_channel in self.news_channels:
            response = requests.get(self.news_channels[self.news_channel])
            data = response.json()
            articles = data.get('articles', [])
            if articles:
                print(f"⭐ Top headlines from {self.news_channel.capitalize()}:")
                for article in articles:
                    print(f"➡️  {article['title']}: {article['url']}")
                reply = input("Let me know if you need any more help with news? (yes/no)\n").lower()
                if reply in self.yes_commands:
                    self.choose_news_channel()
                else:
                    reply = input("Can I help in any other way? (yes/no)\n").lower()
                    if reply in self.yes_commands:
                        reply = input("Let me know other channels that you prefer:\n").lower()
                        self.news_channel = reply
                        self.fetch_news()
                    else:
                        print("Goodbye! Have a nice day.")
            else:
                print("🟥 No articles found for this channel.")
                reply = input("Can I help in any other way? (yes/no)\n").lower()
                if reply in self.yes_commands:
                    reply = input("Let me know other channels that you prefer 👇🏽\n").lower()
                    self.news_channel = reply
                    self.fetch_news()
                else:
                    print("Goodbye! Have a nice day.")
        else:
            self.add_news_channel(self.news_channel, f"https://newsapi.org/v2/top-headlines?sources={self.news_channel}&apiKey={self.api_key}")
            self.fetch_news()

news_bot = NewsBot()
news_bot.greet()

🗞️  Welcome to NewsBot! I can fetch top headlines from different news channels.
⭐ Top headlines from Cnn:
➡️  How progressive ceasefire advocate Summer Lee defanged the opposition ahead of her primary in Pennsylvania: https://www.cnn.com/2024/04/21/politics/summer-lee-primary-pennsylvania-12/index.html
➡️  Fareed’s take: Biden must show he’s willing to fight on immigration | CNN Politics: https://www.cnn.com/videos/politics/2024/04/21/fareeds-take-gps-2024-election-abortion-immigration-digvid.cnn
➡️  FBI: Anti-Jewish hate crime cases tripled in wake of Oct. 7 Hamas attack | CNN: https://www.cnn.com/videos/us/2024/04/21/fbi-alert-antijewish-threats-passover-bernal-pkg-digvid.cnn
➡️  Supreme Court to debate whether cities can punish people who are homeless: https://www.cnn.com/2024/04/21/politics/supreme-court-debate-cities-punish-homeless/index.html
➡️  The US dollar is strengthening. Here’s what’s driving the rally and what it means for Americans: https://www.cnn.com/2024/04/21/markets